## Model 1b: Single-country policy simulation

The objective of this model-based simulation is to analyse the impact of policy, technology, and commodity changes on consumer price inflation in selected countries. The simulation environment is learnt from real data, after which simulations using synthetic data are used to do policy analysis by manipulating a number of selected variables such as government debt, cellular subscription, gdp growth, and real interest rates in the synthetic data. A secondary purpose of the simulation model is to identify and map the interactions between world-level and country-level indicator variables.

#### Features
------------

Human and technological development indicator timeseries for a country x.

#### Labels
----------

Consumer price inflation levels.

#### Training
------------

Training is done on a feature - single country basis.

### Load and prepare the data

In [ ]:
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
import tensorflow_probability as tfp
import matplotlib.pyplot as plt
from tensorflow import keras

%matplotlib inline

In [ ]:
warnings.filterwarnings('ignore')
pd.options.display.float_format = '{:20,.4f}'.format

In [ ]:
sns.set_style("whitegrid")
sns.set_palette("colorblind")

In [ ]:
tf.__version__

In [ ]:
country = 'France'

#### Load and combine the features and labels

In [ ]:
features_df = pd.read_csv('features/m_one/%s_features.csv' % country, sep=';', header=0)
labels_df = pd.read_csv('features/m_one/labels_interpolated.csv', sep=';', header=0)

In [ ]:
features_df.head()

In [ ]:
labels_df.head()

In [ ]:
combined_df = pd.concat([features_df, labels_df.drop(columns=['date'])], axis=1)

In [ ]:
combined_df.head()

In [ ]:
fig, ax = plt.subplots(figsize=(15,7))
[sns.lineplot(x='date', y=c, markers=True, ax=ax, label=c, data=combined_df) for c in list([country, 'lending interest rate', 'real interest rate', 'inflation', 'gross domestic savings', 'government debt service'])]

xticks=ax.xaxis.get_major_ticks()
for i in range(len(xticks)):
    if i % 12 == 1:
        xticks[i].set_visible(True)
    else:
        xticks[i].set_visible(False)

ax.set_xticklabels(combined_df['date'], rotation=45);

In [ ]:
combined_df.columns

### Prepare the country features

In [ ]:
base_feature_df = combined_df[['date', 'bank capital to assets ratio', 'bank nonperforming loans', 'cereal yield',
                               'energy imports', 'food exports', 'high-tech exports', 'inflation',
                               'lending interest rate', 'life expectancy', 'population density', 'real interest rate',
                               'broad money', 'exports of goods and services', 'gross domestic savings',
                               'high-tech value added', 'household consumption expenditure',
                               'imports of goods and services', 'listed companies', 'manufacturing value added',
                               'r and d spend', 'services trade', 'trade', 'government debt service',
                               'government interest payments external debt', 'government tax revenue', 'birth deaths',
                               'broadband subscriptions', 'electricity access', 'co2 emissions',
                               'electricity consumption', 'mobile subscriptions', 'newborns', 'overweight',
                               'rural population', 'urban population', country]]

In [ ]:
base_feature_df.to_csv('features/m_one/combined_country_level_%s.csv' % country.lower(), sep=',', index=False)

In [ ]:
base_feature_df['label'] = base_feature_df[country].shift(periods=1)
base_df = base_feature_df.drop(country, axis=1).fillna(0.00);
base_df.set_index('date')

In [ ]:
num_obs = len(base_df)
num_cols = len(base_df.columns)
num_features = len(base_df.columns) - 1

### Model iterations
---------------------

### Exploration 0

**ARIMA** fitted on the real data.

In [ ]:
from statsmodels.tsa.arima_model import ARIMA
from sklearn.metrics import mean_squared_error

In [ ]:
ar_params = {
    'lag': 4,
    'difference': 2,
    'moving_average': 1
}

ARIMA dataprep

In [ ]:
exo_cols = ['co2 emissions', 'rural population', 'electricity consumption', 'lending interest rate']

ar_endo = base_df['label'].values
ar_exo = base_df[exo_cols].values

In [ ]:
fig, ax = plt.subplots(figsize=(15,7))
[sns.lineplot(x='date', y=c, markers=True, ax=ax, label='cpi in %s' % c, data=combined_df) for c in list([country])]

xticks=ax.xaxis.get_major_ticks()
for i in range(len(xticks)):
    if i % 12 == 1:
        xticks[i].set_visible(True)
    else:
        xticks[i].set_visible(False)

ax.set_xticklabels(combined_df['date'], rotation=45);

In [ ]:
len(ar_endo)

In [ ]:
ar_endo_train, ar_endo_test = ar_endo[0:550], ar_endo[551:696]
ar_exo_train, ar_exo_test = ar_exo[0:550], ar_exo[551:696]

In [ ]:
ar_exo_test[0]

Fit the ARIMA model

In [ ]:
arima = ARIMA(ar_endo, order=(ar_params['lag'], ar_params['difference'], ar_params['moving_average']), exog=ar_exo)

In [ ]:
arima_fitted = arima.fit()

In [ ]:
arima_fitted.summary()

#### Evaluate the ARIMA predictions

In [ ]:
preds = []
obs = []
hist = [x for x in ar_endo_train]
exo_hist = [x for x in ar_exo_train]

for t in range(len(ar_endo_test)):
    m = ARIMA(hist, order=(ar_params['lag'], ar_params['difference'], ar_params['moving_average']))
    m_fit = m.fit()
    yhat = m_fit.forecast()[0][0]
    preds.append(yhat)
    hist.append(ar_endo_test[t])
    exo_hist.append(ar_exo_test[t])
    if t % 50 == 0:
        print('obs: %s, pred: %s' % (ar_endo_test[t], yhat))

In [ ]:
predictions = list(map(lambda x: 0.00 if np.isnan(x) else x, preds))

In [ ]:
mean_squared_error(ar_endo_test, predictions)

In [ ]:
plt.plot(ar_endo_test)
plt.plot(preds, color='green')

### Exploration 1

**Multivariate LSTM** fitted on the real data, see https://machinelearningmastery.com/multivariate-time-series-forecasting-lstms-keras/
- Activation function: Leaky ReLU.
- Loss function: mean squared error.
- Optimizer: adam.
- Num observations source dataset: 684 (using lagshift, 1960-2016 inclusive monthly)
- Num sequences (@ sequence length 6): 116.
- Batch size: 4-8 sequences (although `size=48` would lead to more stable training)

In [ ]:
from keras import Sequential
from keras.layers import LSTM, Dense, LeakyReLU, TimeDistributed
from keras.optimizers import Adam
from sklearn.metrics import mean_squared_error

In [ ]:
lstm_params = {
   'sequence_length': 4,
   'batch_size': 8,
   'num_epochs': 600,
   'num_units': 128,
   'lrelu_alpha': 0.3
}

#### LSTM features

In [ ]:
features = []
labels = []

for i in range(int(num_obs / lstm_params['sequence_length'])):
    labels_df = base_df['label']
    labels.append(labels_df[i:(i+lstm_params['sequence_length'])].values[-1:])
    features.append(base_df[i:(i+lstm_params['sequence_length'])].values)

In [ ]:
lstm_train_X = np.asarray(features[0:100])
lstm_train_X = lstm_train_X.reshape((lstm_train_X.shape[0], lstm_params['sequence_length'], num_cols))
lstm_train_y = np.asarray(labels[0:100])
lstm_train_y = lstm_train_y.reshape((lstm_train_y.shape[0]))

In [ ]:
lstm_test_X = np.asarray(features[100:])
lstm_test_X = lstm_test_X.reshape((lstm_test_X.shape[0], lstm_params['sequence_length'], num_cols))
lstm_test_y = np.asarray(labels[100:])
lstm_test_y = lstm_test_y.reshape((lstm_test_y.shape[0]))

In [ ]:
X = np.asarray(features)
X = X.reshape((X.shape[0], lstm_params['sequence_length'], num_cols))
y = np.asarray(labels)
y = y.reshape((y.shape[0], 1))

In [ ]:
print('X: %s, y: %s' % (X.shape, y.shape))

#### Model: LSTM

In [ ]:
model = Sequential()
model.add(LSTM(lstm_params['num_units'], input_shape=(lstm_params['sequence_length'], num_cols)))
model.add(Dense(1, activation=LeakyReLU(alpha=lstm_params['lrelu_alpha'])))
model.compile(loss='mse', optimizer='adam')
model.summary()

In [ ]:
from keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='loss', mode='min', patience=8)

In [ ]:
train_run = model.fit(lstm_train_X, lstm_train_y, epochs=lstm_params['num_epochs'],
                      batch_size=lstm_params['batch_size'], callbacks=[early_stopping])

In [ ]:
plt.plot(train_run.history['loss'], label='train')
plt.legend()
plt.show()

##### Evaluate model performance

In [ ]:
model.evaluate(lstm_test_X, lstm_test_y)

In [ ]:
yhat = model.predict(lstm_test_X)

In [ ]:
plt.figure(figsize=(15,7))
plt.plot(lstm_test_y, label='observed')
plt.plot(yhat, label='predicted')
plt.legend()
plt.title('Observed versus predicted values for consumer price inflation in %s' % country)
plt.show()

In [ ]:
print('rmse: %s\nmean observed: %s\nmean predicted: %s' % (np.sqrt(mean_squared_error(lstm_test_y, yhat)),
                                                           np.mean(lstm_test_y), np.mean(yhat)))

## Exploration 2
--------------------

**GAN** to generate training data, **LSTM** trained on generated data validated on the real data.

### Conditional GAN for policy-constrained timeseries generation

See https://arxiv.org/pdf/1706.02633.pdf.

In [ ]:
from keras.models import Sequential, Model
from keras.layers import Input
from keras.optimizers import Adam
from sklearn.metrics import mean_squared_error

In [ ]:
gan_df = base_df[['label', 'inflation']]
gan_df.shape

In [ ]:
gan_cols = gan_df.shape[1]

In [ ]:
gan_params = {
   'num_epochs': 800,
   'save_interval': 100,
   'sequence_length': 6,
   'num_variables': gan_cols,
   'batch_size': 64,
   'lr': 0.0001 
}

In [ ]:
generator_params = {
   'noise_sigma': 0.3,
   'lstm_units': 128,
   'lstm_dropout': 0.4,
   'gru_units': 64,
   'lr': 0.0001
}

In [ ]:
discriminator_params = {
   'bi_lstm_units': 256,
   'dropout_rate': 0.4,
   'lr': 0.0001
}

#### GAN input sequences

The collated World Bank and IMF data used as input for the data generator and to validate the model trained on generated data.

In [ ]:
gan_features = []
gan_labels = []

for i in range(int(num_obs / gan_params['sequence_length'])):
    gan_labels_df = gan_df['label']
    gan_labels.append(gan_labels_df[i:(i+gan_params['sequence_length'])].values[-1:])
    gan_features.append(gan_df[i:(i+gan_params['sequence_length'])].values)

In [ ]:
real = np.asarray(gan_features)
real = real.reshape((real.shape[0], gan_params['sequence_length'], gan_cols))

In [ ]:
real.shape

#### Generator

In [ ]:
from keras.layers import GaussianNoise, LSTM, Dropout, BatchNormalization, Dense, LocallyConnected2D, GRU, Reshape

In [ ]:
def build_encoder(params):
    gshape = params['sequence_length'], params['num_variables']
    inputs = Input(shape=(gshape))
    
    e = Sequential(name='encoder')
    e.add(LSTM(params['lstm_units'], input_shape=(gshape), return_sequences=True))
    e.add(Dropout(params['lstm_dropout']))
    e.add(GaussianNoise(stddev=params['noise_sigma']))
    e.add(BatchNormalization(axis=2, momentum=0.8, epsilon=0.01))
    e.add(Dense(params['num_variables'], activation='relu'))
    e.summary()
    
    return Model(inputs, e(inputs))

In [ ]:
encoder = build_encoder({**gan_params, **generator_params})

In [ ]:
def build_generator(params):
    gshape = params['sequence_length'], params['num_variables']
    inputs = Input(shape=(gshape))
    
    g = Sequential(name='generator')
    g.add(GRU(params['gru_units'], input_shape=(gshape), return_sequences=True))
    g.add(Dense(params['num_variables'], activation='softmax'))
    g.add(Reshape(target_shape=(gshape)))
    g.summary()
    
    return Model(inputs, g(inputs))

In [ ]:
generator = build_generator({**gan_params, **generator_params})

#### Discriminator

In [ ]:
from keras.layers import Bidirectional, LSTM, Dense, concatenate, Flatten

In [ ]:
def build_discriminator(params):
    dshape = params['sequence_length'], params['num_variables']
    batch_shape = params['batch_size'], params['sequence_length'], params['num_variables']
    
    real = Input(shape=(dshape))
    generated = Input(shape=(dshape))
    inputs = concatenate([generated, real], axis=1)
    
    d = Sequential(name='discriminator')
    d.add(Bidirectional(LSTM(params['bi_lstm_units']), batch_input_shape=(batch_shape)))
    d.add(Dropout(params['dropout_rate']))
    d.add(Dense(1, activation='sigmoid'))
    d.summary()
    return Model([generated, real], d(inputs))

In [ ]:
discriminator = build_discriminator({**gan_params, **discriminator_params})
discriminator.compile(loss='binary_crossentropy', optimizer=Adam(lr=discriminator_params['lr']), metrics=['accuracy'])

#### GAN

Bidirectional generative adversarial network, viz https://arxiv.org/abs/1605.09782.

In [ ]:
def build_gan(encoder, generator, discriminator, params):
    ganshape = params['sequence_length'], params['num_variables']
    discriminator.trainable = False
    
    noise = Input(shape=(ganshape))
    generated = generator(noise)
    
    data = Input(shape=(ganshape))
    encoded = encoder(data)
    
    fake = discriminator([noise, generated])
    real = discriminator([encoded, data])
    
    gan = Model([noise, data], [fake, real], name='gan')
    gan.summary()
    return gan

In [ ]:
gan = build_gan(encoder, generator, discriminator, gan_params)
gan.compile(loss=['kullback_leibler_divergence', 'kullback_leibler_divergence'], 
            optimizer=Adam(lr=generator_params['lr']), metrics=['mse', 'mse'])

In [ ]:
def train_gan(real, batch_size, params):
    g_metrics = []
    d_real_metrics = []
    d_synth_metrics = []
    
    reals = np.ones(batch_size)
    synths = np.zeros(batch_size)
    
    for i in range(params['num_epochs']):
        # create input of real and synthetic data
        random_index = np.random.randint(0, len(real) - batch_size)
        half_real = real[random_index:int(random_index + batch_size)]
        half_synth = np.random.normal(-1.0, 1.0, size=[batch_size, params['sequence_length'], real.shape[2]])
        
        # apply generator and encoder
        generated = generator.predict(half_synth)
        encoded = encoder.predict(half_real)
        
        # train discriminator
        d_real = discriminator.train_on_batch([encoded, half_real], reals)
        d_synth = discriminator.train_on_batch([half_synth, generated], synths)
                                                            
        # train gan
        gen_ = gan.train_on_batch([half_synth, half_real], [reals, synths])
        if i % 100 == 0:
            print('Epoch %s losses: discriminator real: %.4f%%, discriminator synth: %.4f%%, generator: %.4f%%' % 
                  (i, d_real[0], d_synth[0], gen_[0]))
        
        d_real_metrics.append(d_real)
        d_synth_metrics.append(d_synth)
        g_metrics.append(gen_)
    return d_real_metrics, d_synth_metrics, g_metrics

In [ ]:
d_r_metrics, d_s_metrics, g_metrics = train_gan(real, gan_params['batch_size'], gan_params)

In [ ]:
plt.figure(figsize=(15,7))
plt.plot([metrics[0] for metrics in d_r_metrics], label='discriminator loss on reals')
plt.plot([metrics[0] for metrics in d_s_metrics], label='discriminator loss on synths')
plt.plot([metrics[0] for metrics in g_metrics], label='generator loss')
plt.legend()
plt.title('GAN losses')
plt.show()

In [ ]:
plt.figure(figsize=(15,7))
plt.plot([metrics[1] for metrics in d_r_metrics], label='discriminator accuracy reals')
plt.plot([metrics[1] for metrics in d_s_metrics], label='discriminator accuracy synths')
plt.plot([metrics[1] for metrics in g_metrics], label='generator mean average error')
plt.legend()
plt.title('GAN performance metrics')
plt.show()

In [ ]:
generated_y = generator.predict(np.random.rand(num_obs, gan_params['sequence_length'], gan_cols))[:,-1,-1]
gan_y = gan_df['label'].values

In [ ]:
plt.figure(figsize=(15,7))
plt.plot(gan_y, label='observed cpi')
plt.plot(generated_y, label='gan-generated cpi')
plt.legend()
plt.title('Observed versus GAN-generated values for consumer price inflation in %s' % country)
plt.show()

In [ ]:
print('rmse: %s\nmean observed: %s\nmean generated: %s' % (np.sqrt(mean_squared_error(gan_y, generated_y)),
                                                           np.mean(gan_y), np.mean(generated_y)))

## Exploration 3
--------------------

**Sequence transformer network** to generate training data, **LSTM** trained on generated data validated on the real data. See https://arxiv.org/abs/1808.06725